Importing the necessary libraries

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers, applications
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array, load_img
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt

Mounting the dataset to the workspace

In [2]:
import azureml.core
from azureml.core import Workspace, Dataset
connect_str = os.getenv('DefaultEndpointsProtocol=https;AccountName=cakes;AccountKey=0BYHrONq+9pwMQafmGe+Oq4UqJ8lA2F41DZhw5lZSCRkIyddpxEh+bzk/msx88A70ihGFwrIcbcdINHUUmzskw==;EndpointSuffix=core.windows.net')
connect_str

url= Dataset.File.from_files('https://cakes.blob.core.windows.net/train')
ws= Workspace.from_config()
data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

url.download(data_folder, overwrite=True)

cake_file_dataset = url.register(workspace=ws,
                                name='train',
                                description='training',
                                create_new_version=True)

In [3]:
from azureml.core import Dataset
connect_str = os.getenv('DefaultEndpointsProtocol=https;AccountName=cakes;AccountKey=0BYHrONq+9pwMQafmGe+Oq4UqJ8lA2F41DZhw5lZSCRkIyddpxEh+bzk/msx88A70ihGFwrIcbcdINHUUmzskw==;EndpointSuffix=core.windows.net')
connect_str

url= Dataset.File.from_files('https://cakes.blob.core.windows.net/test')

data_folder2 = os.path.join(os.getcwd(), 'data2')
os.makedirs(data_folder2, exist_ok=True)

url.download(data_folder2, overwrite=True)

cakess_file_dataset = url.register(workspace=ws,
                                                 name='test2',
                                                 description='testing',
                                                 create_new_version=True)

Labeling the data: the data is labeled based on the name of the folder

In [4]:
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)


train_dataset = train.flow_from_directory('data/train',
                                          target_size=(150,150),
                                          batch_size = 32,
                                          class_mode = 'sparse')
                                         
test_dataset = test.flow_from_directory('data2/test',
                                          target_size=(150,150),
                                          batch_size =32,
                                          class_mode = 'sparse')

Found 399 images belonging to 5 classes.
Found 100 images belonging to 5 classes.


In [5]:
test_dataset.class_indices

{'BundtCake': 0,
 'ChocolateCupCake': 1,
 'OneStepCake': 2,
 'TwoStepCake': 3,
 'VanillaCupCake': 4}

In [6]:
model = keras.Sequential()
model.add(keras.Input(shape=(150, 150, 3)))
model.add(layers.Conv2D(4, (3, 3), activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.MaxPooling2D((2, 2), strides=2))
model.add(layers.Conv2D(4, (3, 3), activation='relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.MaxPooling2D((2, 2), strides=2))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation= None ))

model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

In [7]:
 model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 4)       112       
_________________________________________________________________
dropout (Dropout)            (None, 148, 148, 4)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 4)         148       
_________________________________________________________________
dropout_1 (Dropout)          (None, 72, 72, 4)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 4)         0         
_________________________________________________________________
flatten (Flatten)            (None, 5184)              0

In [8]:
model.fit_generator(train_dataset,
        #steps_per_epoch = 5,
         epochs = 20,
         validation_data = test_dataset
         )

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 13 steps, validate for 4 steps
Epoch 1/20
13/13 [==============================] - 22s 2s/step - loss: 2.1565 - sparse_categorical_accuracy: 0.2180 - val_loss: 1.7660 - val_sparse_categorical_accuracy: 0.1800
Epoch 2/20
13/13 [==============================] - 19s 1s/step - loss: 1.5896 - sparse_categorical_accuracy: 0.2907 - val_loss: 1.9785 - val_sparse_categorical_accuracy: 0.2100
Epoch 3/20
13/13 [==============================] - 19s 1s/step - loss: 1.5256 - sparse_categorical_accuracy: 0.3534 - val_loss: 1.7626 - val_sparse_categorical_accuracy: 0.2900
Epoch 4/20
13/13 [==============================] - 19s 1s/step - loss: 1.4099 - sparse_categorical_accuracy: 0.4135 - val_loss: 1.6286 - val_sparse_categorical_accuracy: 0.3200
Epoch 5/20
13/13 [==============================] - 19s 1s/step - loss: 1.2714 - sparse_categorical_accuracy: 0.4912 - va

In [9]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(train_dataset)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(test_dataset[1])
print("predictions shape:", predictions.shape)

Evaluate on test data
  ...
    to  
  ['...']
13/13 [==============================] - 16s 1s/step - loss: 0.6696 - sparse_categorical_accuracy: 0.9273
test loss, test acc: [0.6695881967361157, 0.9273183]
Generate predictions for 3 samples
predictions shape: (32, 10)


In [10]:
image_path = 'data2/test/BundtCake/100.jpg'  
img_width, img_height = 150, 150

# orig = cv2.imread(image_path)  

print("[INFO] loading and preprocessing image...")  
image = load_img(image_path, target_size=(img_width, img_height))  
image2 = img_to_array(image)

# important! otherwise the predictions will be '0'  
image3 = image2 / 255  

image4 = np.expand_dims(image2, axis=0)

# build the VGG16 network  
model = applications.VGG16(include_top=False, weights='imagenet')  

# get the bottleneck prediction from the pre-trained VGG16 model  
bottleneck_prediction = model.predict(image4)

generator_top = test_dataset

n_classes = len(generator_top.class_indices)

# build top model  
model = keras.Sequential() 
model.add(layers.Flatten(input_shape=bottleneck_prediction.shape[1:]))  
model.add(layers.Dense(256, activation='relu'))  
model.add(layers.Dropout(0.5))  
model.add(layers.Dense(n_classes, activation=None))  

# model.load_weights(top_model_weights_path)  

# use the bottleneck prediction on the top model to get the final classification  
class_predicted = model.predict_classes(bottleneck_prediction) 

inID = class_predicted[0]  

class_dictionary = generator_top.class_indices  

inv_map = {v: k for k, v in class_dictionary.items()}  

label = inv_map[inID]  

# get the prediction label  
print("Image ID: {}, Label: {}".format(inID, label)) 

[INFO] loading and preprocessing image...
Image ID: 3, Label: TwoStepCake
